In [ ]:
# ============================
# 1. Install Dependencies
# ============================
!pip install -q timm torchmetrics tqdm

# ============================
# 2. Imports
# ============================
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

from timm import create_model
from torchmetrics.classification import Accuracy

# ============================
# 3. Configurations
# ============================
dir="/content/drive/MyDrive/jaundice_dataset"

CSV_PATH =f"{dir}/chd_jaundice_published_2.csv"    # path to your CSV
IMAGE_DIR = f"{dir}/images"           # folder containing all images

BATCH_SIZE = 32
EPOCHS = 10
LR = 1e-4
IMG_SIZE = 224
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ============================
# 4. Read CSV and Prepare Data
# ============================
df = pd.read_csv(CSV_PATH)
print("✅ CSV Loaded. Total samples:", len(df))

# Ensure label column
if "Treatment" not in df.columns:
    raise ValueError("❌ 'Treatment' column not found! Add label column 0/1 in CSV.")
df["label"] = df["Treatment"].astype(int)

# Build full image paths
df["image_path"] = df["image_idx"].apply(lambda x: os.path.join(IMAGE_DIR, x))
if not all(df["image_path"].apply(os.path.exists)):
    raise FileNotFoundError("❌ Some image files not found in IMAGE_DIR.")

# Split into train/val/test
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
train_df, val_df  = train_test_split(train_df, test_size=0.1, stratify=train_df["label"], random_state=42)

print(f"📊 Split sizes -> Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# ============================
# 5. Dataset Class
# ============================
class JaundiceDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, "image_path"]
        label = self.df.loc[idx, "label"]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label

# ============================
# 6. Transforms and Loaders
# ============================
train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_loader = DataLoader(JaundiceDataset(train_df, train_tfms),
                          batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(JaundiceDataset(val_df, val_tfms),
                        batch_size=BATCH_SIZE)
test_loader = DataLoader(JaundiceDataset(test_df, val_tfms),
                         batch_size=BATCH_SIZE)

# ============================
# 7. Model Setup (ViT Tiny)
# ============================
model = create_model('vit_tiny_patch16_224', pretrained=True)
model.head = nn.Linear(model.head.in_features, 2)  # Binary classification
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
train_acc = Accuracy(task="binary").to(DEVICE)
val_acc = Accuracy(task="binary").to(DEVICE)

# ============================
# 8. Training Loop
# ============================
for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0
    train_acc.reset()

    # ---- TRAINING PHASE ----
    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS} - Training"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        train_acc.update(preds, labels)

    avg_loss = total_loss / len(train_loader)
    train_accuracy = train_acc.compute().item()

    # ---- VALIDATION PHASE ----
    model.eval()
    val_loss = 0.0
    val_acc.reset()

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            val_acc.update(preds, labels)

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = val_acc.compute().item()

    print(f"🧠 Epoch {epoch} | "
          f"Train Loss: {avg_loss:.4f}, Acc: {train_accuracy:.4f} | "
          f"Val Loss: {avg_val_loss:.4f}, Acc: {val_accuracy:.4f}")

# ============================
# 9. Test Evaluation
# ============================
model.eval()
test_acc = Accuracy(task="binary").to(DEVICE)
with torch.no_grad():
    for imgs, labels in tqdm(test_loader, desc="Testing"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        preds = torch.argmax(model(imgs), dim=1)
        test_acc.update(preds, labels)
final_acc = test_acc.compute().item()
print(f"✅ Final Test Accuracy: {final_acc:.4f}")

# ============================
# 10. Save Model
# ============================
torch.save(model.state_dict(), "vit_jaundice.pth")
print("✅ Model saved as vit_jaundice.pth")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 12.4 MB/s eta 0:00:00
✅ CSV Loaded. Total samples: 2235
📊 Split sizes -> Train: 1609, Val: 179, Test: 447


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

Epoch 1/10 - Training: 100%|██████████| 51/51 [14:13<00:00, 16.73s/it]


🧠 Epoch 1 | Train Loss: 0.8315, Acc: 0.5631 | Val Loss: 0.6879, Acc: 0.5810


...

🧠 Epoch 9 | Train Loss: 0.3764, Acc: 0.8254 | Val Loss: 0.5185, Acc: 0.7598


Epoch 10/10 - Training: 100%|██████████| 51/51 [04:39<00:00,  5.49s/it]


🧠 Epoch 10 | Train Loss: 0.3352, Acc: 0.8645 | Val Loss: 0.4858, Acc: 0.7933


Testing: 100%|██████████| 14/14 [03:05<00:00, 13.23s/it]

✅ Final Test Accuracy: 0.7629
✅ Model saved as vit_jaundice.pth


In [ ]:
# ============================
# 1. Install Dependencies
# ============================
!pip install -q timm torchmetrics tqdm

# ============================
# 2. Imports
# ============================
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

from timm import create_model
from torchmetrics.classification import Accuracy

# ============================
# 3. Configurations
# ============================
DIR = "/content/drive/MyDrive/jaundice_dataset"
CSV_PATH = f"{DIR}/chd_jaundice_published_2.csv"   # path to your CSV
IMAGE_DIR = f"{DIR}/images"                        # folder containing all images

BATCH_SIZE = 32
EPOCHS = 30
LR = 3e-5                 # small LR for transformer
IMG_SIZE = 224
PATIENCE = 5              # early stopping patience
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ============================
# 4. Read CSV and Prepare Data
# ============================
df = pd.read_csv(CSV_PATH)
print("✅ CSV Loaded. Total samples:", len(df))

# ✅ Create label column (Jaundiced = 1, Non-Jaundiced = 0)
if "Treatment" not in df.columns:
    raise ValueError("❌ 'Treatment' column not found in CSV.")
df["label"] = df["Treatment"].astype(int)

# ✅ Construct full image paths
df["image_path"] = df["image_idx"].apply(lambda x: os.path.join(IMAGE_DIR, x))
if not all(df["image_path"].apply(os.path.exists)):
    missing = df[~df["image_path"].apply(os.path.exists)]
    raise FileNotFoundError(f"❌ Missing {len(missing)} image files in {IMAGE_DIR}")

# ✅ Split dataset (stratified)
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
train_df, val_df  = train_test_split(train_df, test_size=0.1, stratify=train_df["label"], random_state=42)
print(f"📊 Dataset Split -> Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# ============================
# 5. Dataset Class
# ============================
class JaundiceDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        img_path = row["image_path"]
        label = row["label"]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label

# ============================
# 6. Transforms & Dataloaders
# ============================

# ⚠️ Keep augmentations mild (color-sensitive task)
train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

train_loader = DataLoader(JaundiceDataset(train_df, train_tfms),
                          batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(JaundiceDataset(val_df, val_tfms),
                        batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(JaundiceDataset(test_df, val_tfms),
                         batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# ============================
# 7. Model Setup (ViT Tiny)
# ============================
model = create_model('vit_tiny_patch16_224', pretrained=True)
model.head = nn.Linear(model.head.in_features, 2)  # binary classification
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
train_acc = Accuracy(task="binary").to(DEVICE)
val_acc = Accuracy(task="binary").to(DEVICE)

# ============================
# 8. Training Loop (with Early Stopping)
# ============================
best_val_acc = 0.0
patience_counter = 0

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0.0
    train_acc.reset()

    # ---- TRAIN ----
    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS} - Training"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        train_acc.update(preds, labels)

    avg_train_loss = total_loss / len(train_loader)
    train_accuracy = train_acc.compute().item()

    # ---- VALIDATE ----
    model.eval()
    val_loss = 0.0
    val_acc.reset()
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            val_acc.update(preds, labels)

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = val_acc.compute().item()

    print(f"🧠 Epoch {epoch:02d} | "
          f"Train Loss: {avg_train_loss:.4f}, Acc: {train_accuracy:.4f} | "
          f"Val Loss: {avg_val_loss:.4f}, Acc: {val_accuracy:.4f}")

    # ---- EARLY STOP ----
    if val_accuracy > best_val_acc:
        best_val_acc = val_accuracy
        patience_counter = 0
        torch.save(model.state_dict(), "best_vit_jaundice.pth")
        print("✅ Saved new best model")
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print("⏹️ Early stopping triggered.")
            break

# ============================
# 9. Test Evaluation
# ============================
print("\n🔍 Evaluating on Test Set...")
model.load_state_dict(torch.load("best_vit_jaundice.pth"))
model.eval()
test_acc = Accuracy(task="binary").to(DEVICE)

with torch.no_grad():
    for imgs, labels in tqdm(test_loader, desc="Testing"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        preds = torch.argmax(model(imgs), dim=1)
        test_acc.update(preds, labels)

final_acc = test_acc.compute().item()
print(f"✅ Final Test Accuracy: {final_acc:.4f}")

# ============================
# 10. Save Final Model
# ============================
torch.save(model.state_dict(), "vit_jaundice_final.pth")
print("💾 Model saved as vit_jaundice_final.pth")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 19.0 MB/s eta 0:00:00
✅ CSV Loaded. Total samples: 2235
📊 Dataset Split -> Train: 1609, Val: 179, Test: 447


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

Epoch 1/30 - Training: 100%|██████████| 51/51 [05:34<00:00,  6.57s/it]


🧠 Epoch 01 | Train Loss: 0.7298, Acc: 0.6196 | Val Loss: 0.5972, Acc: 0.6816
✅ Saved new best model


...

🧠 Epoch 11 | Train Loss: 0.2394, Acc: 0.9030 | Val Loss: 0.4849, Acc: 0.8212


Epoch 12/30 - Training: 100%|██████████| 51/51 [05:08<00:00,  6.06s/it]


🧠 Epoch 12 | Train Loss: 0.2399, Acc: 0.8943 | Val Loss: 0.5093, Acc: 0.7877
⏹️ Early stopping triggered.

🔍 Evaluating on Test Set...


Testing: 100%|██████████| 14/14 [01:17<00:00,  5.53s/it]

✅ Final Test Accuracy: 0.8076
💾 Model saved as vit_jaundice_final.pth


In [ ]:
import os
import cv2
from tqdm import tqdm
import pandas as pd

# Paths
base_dir = "/content/drive/MyDrive/jaundice_dataset"
IMG_DIR = f"{base_dir}/images"
CSV_PATH = f"{base_dir}/chd_jaundice_published_2.csv"
OUT_DIR = f"{base_dir}/skin_patches"
os.makedirs(OUT_DIR, exist_ok=True)

# Read CSV
df = pd.read_csv(CSV_PATH)

# Define skin crop coordinates (replace these with correct ones)
# Format: (y1, y2, x1, x2)
SKIN_REGION = (150, 350, 150, 350)

# Crop and save
records = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    filename = row["image_idx"]
    label = row["Treatment"]

    img_path = os.path.join(IMG_DIR, filename)
    if not os.path.exists(img_path):
        continue

    img = cv2.imread(img_path)
    if img is None:
        continue

    y1, y2, x1, x2 = SKIN_REGION
    h, w = img.shape[:2]
    y1, y2 = min(y1, h), min(y2, h)
    x1, x2 = min(x1, w), min(x2, w)

    skin_patch = img[y1:y2, x1:x2]
    out_path = os.path.join(OUT_DIR, filename)
    cv2.imwrite(out_path, skin_patch)

    records.append({
        "image": filename,
        "label": int(label)
    })

# Save new CSV for cropped patches
df_new = pd.DataFrame(records)
df_new.to_csv(f"{base_dir}/skin_patches.csv", index=False)
print("✅ Cropped patches saved & CSV created:", len(df_new))


100%|██████████| 2235/2235 [22:40<00:00,  1.64it/s]


✅ Cropped patches saved & CSV created: 2235


In [ ]:
!pip install -q timm torchmetrics tqdm

import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

from timm import create_model
from torchmetrics.classification import Accuracy

# Paths
dir = "/content/drive/MyDrive/jaundice_dataset"
CSV_PATH = f"{dir}/skin_patches.csv"
IMAGE_DIR = f"{dir}/skin_patches"

# Config
BATCH_SIZE = 32
EPOCHS = 30
LR = 3e-5
IMG_SIZE = 224
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Read CSV
df = pd.read_csv(CSV_PATH)
print("✅ Loaded cropped skin dataset:", len(df))

# Split dataset
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, stratify=train_df["label"], random_state=42)
print(f"📊 Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# Dataset class
class SkinDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = os.path.join(IMAGE_DIR, self.df.loc[idx, "image"])
        label = self.df.loc[idx, "label"]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label

# Transforms
train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# DataLoaders
train_loader = DataLoader(SkinDataset(train_df, train_tfms), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(SkinDataset(val_df, val_tfms), batch_size=BATCH_SIZE)
test_loader = DataLoader(SkinDataset(test_df, val_tfms), batch_size=BATCH_SIZE)

# Model (ViT Tiny)
model = create_model('vit_tiny_patch16_224', pretrained=True)
model.head = nn.Linear(model.head.in_features, 2)
model = model.to(DEVICE)

# Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
train_acc = Accuracy(task="binary").to(DEVICE)
val_acc = Accuracy(task="binary").to(DEVICE)

# Training Loop
for epoch in range(1, EPOCHS+1):
    model.train()
    total_loss = 0.0
    train_acc.reset()

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        train_acc.update(preds, labels)

    model.eval()
    val_loss, val_acc_val = 0.0, 0.0
    val_acc.reset()

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = outputs.argmax(dim=1)
            val_acc.update(preds, labels)

    print(f"🧠 Epoch {epoch} | Train Loss: {total_loss/len(train_loader):.4f}, "
          f"Train Acc: {train_acc.compute():.4f} | Val Loss: {val_loss/len(val_loader):.4f}, "
          f"Val Acc: {val_acc.compute():.4f}")

# Test Evaluation
model.eval()
test_acc = Accuracy(task="binary").to(DEVICE)
with torch.no_grad():
    for imgs, labels in tqdm(test_loader, desc="Testing"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        preds = model(imgs).argmax(dim=1)
        test_acc.update(preds, labels)
print(f"✅ Final Test Accuracy: {test_acc.compute():.4f}")

# Save model
torch.save(model.state_dict(), f"{dir}/vit_skin_jaundice.pth")
print("✅ Model saved as vit_skin_jaundice.pth")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 13.1 MB/s eta 0:00:00
✅ Loaded cropped skin dataset: 2235
📊 Train: 1609, Val: 179, Test: 447


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

Epoch 1/30: 100%|██████████| 51/51 [05:05<00:00,  5.99s/it]


🧠 Epoch 1 | Train Loss: 0.7300, Train Acc: 0.6085 | Val Loss: 0.5764, Val Acc: 0.6760


...

🧠 Epoch 29 | Train Loss: 0.1946, Train Acc: 0.9180 | Val Loss: 0.6699, Val Acc: 0.7542


Epoch 30/30: 100%|██████████| 51/51 [04:49<00:00,  5.67s/it]


🧠 Epoch 30 | Train Loss: 0.1566, Train Acc: 0.9335 | Val Loss: 0.7311, Val Acc: 0.7654


Testing: 100%|██████████| 14/14 [00:28<00:00,  2.05s/it]

✅ Final Test Accuracy: 0.7494
✅ Model saved as vit_skin_jaundice.pth


In [ ]:
# ==========================
# 1. Install Dependencies
# ==========================
!pip install -q timm torchmetrics tqdm pandas

# ==========================
# 2. Imports
# ==========================
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

from timm import create_model
from torchmetrics.classification import Accuracy

# ==========================
# 3. Config
# ==========================
DATA_DIR = "/content/drive/MyDrive/jaundice_dataset"
CSV_PATH = f"{DATA_DIR}/chd_jaundice_published_2.csv"
IMAGE_DIR = f"{DATA_DIR}/images"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 40
LR = 3e-5
PATIENCE = 7  # for early stopping

# ==========================
# 4. Load and Prepare Data
# ==========================
df = pd.read_csv(CSV_PATH)
df = df.rename(columns={"image_idx": "image", "blood(mg/dL)": "bilirubin"})
df["image_path"] = df["image"].apply(lambda x: os.path.join(IMAGE_DIR, x))
df = df[df["image_path"].apply(os.path.exists)]

def categorize_bilirubin(x):
    if x < 12: return 0
    elif x < 17: return 1
    elif x < 25: return 2
    else: return 3

df["label"] = df["bilirubin"].apply(categorize_bilirubin)

train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, stratify=train_df["label"], random_state=42)

print(f"✅ Data Prepared: {len(df)} images")
print(f"📊 Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

# ==========================
# 5. Dataset Class
# ==========================
def crop_center(img, crop_size=200):
    w, h = img.size
    left = (w - crop_size) // 2
    top = (h - crop_size) // 2
    return img.crop((left, top, left + crop_size, top + crop_size))

class JaundiceDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img = Image.open(self.df.loc[idx, "image_path"]).convert("RGB")
        img = crop_center(img, crop_size=200)
        label = self.df.loc[idx, "label"]
        if self.transform:
            img = self.transform(img)
        return img, label

# ==========================
# 6. Transforms & Dataloaders
# ==========================
train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.85, 1.0)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_loader = DataLoader(JaundiceDataset(train_df, train_tfms), batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(JaundiceDataset(val_df, val_tfms), batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(JaundiceDataset(test_df, val_tfms), batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# ==========================
# 7. Model (ViT Base)
# ==========================
NUM_CLASSES = 4
model = create_model("vit_tiny_patch16_224", pretrained=True)
model.head = nn.Linear(model.head.in_features, NUM_CLASSES)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

train_acc = Accuracy(task="multiclass", num_classes=NUM_CLASSES).to(DEVICE)
val_acc = Accuracy(task="multiclass", num_classes=NUM_CLASSES).to(DEVICE)

# ==========================
# 8. Train Loop with Early Stopping
# ==========================
best_val_acc = 0
no_improve_epochs = 0

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0
    train_acc.reset()

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()

        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            outputs = model(imgs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        train_acc.update(preds, labels)

    scheduler.step()

    # Validation
    model.eval()
    val_loss = 0
    val_acc.reset()

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                outputs = model(imgs)
                loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            val_acc.update(preds, labels)

    train_accuracy = train_acc.compute()
    val_accuracy = val_acc.compute()

    print(f"🧠 Epoch {epoch}: "
          f"TrainLoss={total_loss/len(train_loader):.4f}, TrainAcc={train_accuracy:.3f}, "
          f"ValLoss={val_loss/len(val_loader):.4f}, ValAcc={val_accuracy:.3f}")

    # Early Stopping
    if val_accuracy > best_val_acc:
        best_val_acc = val_accuracy
        no_improve_epochs = 0
        torch.save(model.state_dict(), f"{DATA_DIR}/vit_best_jaundice_model.pth")
    else:
        no_improve_epochs += 1
        if no_improve_epochs >= PATIENCE:
            print("⏹️ Early stopping triggered!")
            break

print(f"✅ Best Validation Accuracy: {best_val_acc:.3f}")

# ==========================
# 9. Test Evaluation
# ==========================
model.load_state_dict(torch.load(f"{DATA_DIR}/vit_best_jaundice_model.pth"))
model.eval()
test_acc = Accuracy(task="multiclass", num_classes=NUM_CLASSES).to(DEVICE)

with torch.no_grad():
    for imgs, labels in tqdm(test_loader, desc="Testing"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        preds = torch.argmax(model(imgs), dim=1)
        test_acc.update(preds, labels)

print(f"✅ Final Test Accuracy: {test_acc.compute():.3f}")


✅ Data Prepared: 2235 images
📊 Train: 1609 | Val: 179 | Test: 447


model.safetensors:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

/tmp/ipython-input-1483694397.py:120: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
Epoch 1/40:   0%|          | 0/51 [00:00<?, ?it/s]/tmp/ipython-input-1483694397.py:140: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
Epoch 1/40: 100%|██████████| 51/51 [00:18<00:00,  2.78it/s]
/tmp/ipython-input-1483694397.py:162: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


🧠 Epoch 1: TrainLoss=1.0650, TrainAcc=0.515, ValLoss=0.9882, ValAcc=0.536


Epoch 2/40: 100%|██████████| 51/51 [00:19<00:00,  2.59it/s]


...

🧠 Epoch 22: TrainLoss=0.4675, TrainAcc=0.804, ValLoss=1.0560, ValAcc=0.581


Epoch 23/40: 100%|██████████| 51/51 [00:18<00:00,  2.72it/s]


🧠 Epoch 23: TrainLoss=0.4411, TrainAcc=0.814, ValLoss=1.1555, ValAcc=0.570
⏹️ Early stopping triggered!
✅ Best Validation Accuracy: 0.659


Testing: 100%|██████████| 14/14 [00:03<00:00,  3.71it/s]

✅ Final Test Accuracy: 0.658
